<a href="https://colab.research.google.com/github/akshatshah91/Game-AI/blob/master/Monte_Carlo_Tree_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [139]:
import gym
import random
import math
import numpy as np
import copy
import matplotlib.pyplot as plt
from IPython.display import clear_output
from collections import deque
import sys

In [105]:
class Tree:
  def __init__(self, env, actionSize):
    self.env = copy.deepcopy(env)
    self.parent = None
    self.action = None
    self.children = {}
    for x in range(actionSize):
      self.children[x] = (False, None)
    self.state = None
    self.gameOver = None
    self.actionSize = actionSize
    self.visited = 0
    self.won = 0
  
  def isFullyExpanded(self):
    for x in range(self.actionSize):
      if self.children[x][0] is False:
        return False
    return True

  def chooseUntriedAction(self):
    actions = []
    for x in range(self.actionSize):
      if self.children[x][0] is False:
        actions.append(x)
    return random.choice(actions)
  
  def updateChildren(self,action, child):
    self.children[action] = (True, child)

In [141]:
def MCTS(env, s, obervationSize, actionSize, loops=4000):
  root = Tree(env, actionSize)
  root.state = s
  for x in range(loops):
    v1 = treePolicy(root)
    reward = defaultPolicy(v1)
    backup(v1, reward)
  return bestChild(root).action

def treePolicy(v):
  while not v.gameOver:
    if not v.isFullyExpanded():
      return expand(v)
    v = bestChild(v)
  return v

def expand(v):
  action = v.chooseUntriedAction()
  v1 = Tree(v.env, actionSize)
  v1.parent = v
  v1.action = action
  v1.state,v1.reward,v1.gameOver,_ = v1.env.step(action)
  v.updateChildren(action, v1)
  return v1

def bestChild(v):
  c = None
  val = float('-inf')
  for _,(_,child) in v.children.items():
    if child is not None:
      calc = ((child.won/child.visited) + math.sqrt((2*math.log(v.visited))/child.visited))
      if calc > val:
        c = child
        val = calc
  return c

def defaultPolicy(v):
  state = copy.copy(v.state)
  env = copy.deepcopy(v.env)
  gameOver = v.gameOver
  reward = 0
  while not gameOver:
    action = np.random.randint(v.actionSize)
    state,r,gameOver,_ = env.step(action)
    reward += r
  return reward

def backup(v, r):
  while v is not None:
    v.visited += 1
    v.won = max(v.won, r)
    v = v.parent

In [142]:
env = gym.make("Taxi-v3")
observationSize = env.observation_space.n
actionSize = env.action_space.n
s = env.reset()
env.render()
gameOver = False
reward = 0
action = None
while not gameOver or reward < -100:
  action = MCTS(env, s, observationSize, actionSize, loops=500)
  s,r,gameOver,_ = env.step(action)
  reward += r
  env.render()
print(reward)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)


KeyboardInterrupt: ignored